In [13]:
!pip install -q pandas numpy tqdm requests python-dateutil earthengine-api google-cloud-storage google-api-python-client


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import os

In [3]:
base_path = "data/processed/"

df= pd.read_csv(os.path.join(base_path, "analysis_panel.csv"))
print("Analysis panel shape:", df.shape)

# df = pd.read_csv(base_path)
df.drop(columns=['first_treatment_year'], inplace = True)

df['treated'] = df['treated'].astype(int)
display(df.head())


Analysis panel shape: (39947, 11)


,grid_id,year,species,n_occurrences,occupancy,treated,treatment_intensity,lon_center,lat_center,area_km2
0,g_0_321,2018,Chelonia mydas,1,1,0,0.0,60.05,22.15,133.794339
1,g_0_322,2018,Chelonia mydas,1,1,0,0.0,60.05,22.25,133.889669
2,g_0_323,2018,Chelonia mydas,2,1,0,0.0,60.05,22.35,133.985543
3,g_0_324,2018,Chelonia mydas,4,1,0,0.0,60.05,22.45,134.081963
4,g_0_325,2018,Chelonia mydas,2,1,0,0.0,60.05,22.55,134.178931


# Appending Covariates 
The following code block basically adds these 5 features to the df:

- temp_mean_(°C) °C −10 → +35 Mean annual air temperature
- precip_sum_(mm/year) mm/year 0 → 3000 Total yearly rainfall
- humidity_mean_(%) % 20 → 100 Mean relative humidity
- ndvi_mean_(0–1) 0–1 (dimensionless) 0 (bare) → 1 (lush) Vegetation greenness
- human_density_(people/km²) people per km² 0 → 50,000 Population density

each column header follows the max resolution it can use. 

## Apis Used:
OpenMeteo - > temp + precip 
NASA Power -> humidity 
Google Earth Engine -> ndvi + Human density

## To configure Google Earth Engine 

1. Make a google earth engine account at https://code.earthengine.google.com/
2. Make a new project (non commercial) and sound as broke as you can, convince it u dont want to make money off it 
3. run this -> ``` earthengine authenticate ``` in cmd -> this opens ur browser -> login to the acc wit GEE configured
4. below  

In [ ]:
# ============================================================
# 🌍 Improved Climate Covariate Fetcher with Proper Batching
# ============================================================
import requests, time, numpy as np, pandas as pd
from tqdm import tqdm
import json
import os
from collections import defaultdict
import ee

def init_earth_engine():
    """Authenticate and initialize Earth Engine safely (works for Colab & VSCode)."""
    try:
        ee.Initialize()
        print("🌍 Earth Engine initialized (existing credentials).")
    except Exception as e:
        print("🔑 Authenticating Earth Engine...")
        ee.Authenticate()
        ee.Initialize()
        print("✅ Earth Engine successfully initialized!")

# ------------------ 1️⃣ OPEN-METEO (Individual requests with smart caching) ------------------
def fetch_open_meteo_single(lat, lon, year):
    """Fetch temperature & precipitation for a single location."""
    try:
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": f"{year}-01-01",
            "end_date": f"{year}-12-31",
            "daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"],
            "timezone": "UTC",
        }
        r = requests.get(url, params=params, timeout=60)
        
        if r.status_code == 429:
            print("⚠️ Rate limit hit – pausing 60s")
            time.sleep(60)
            return fetch_open_meteo_single(lat, lon, year)
        
        r.raise_for_status()
        js = r.json()
        d = js.get("daily", {})
        
        result = {}
        if d and "temperature_2m_max" in d and "temperature_2m_min" in d:
            temps = [(a + b)/2 for a, b in zip(d["temperature_2m_max"], d["temperature_2m_min"]) 
                     if a is not None and b is not None]
            if temps:
                result["temp_mean_(°C)_(25x25km)"] = np.mean(temps)
        
        if d and "precipitation_sum" in d:
            precip = [p for p in d["precipitation_sum"] if p is not None]
            if precip:
                result["precip_sum_(mm/year)_(25x25km)"] = np.sum(precip)
        
        return result
    except Exception as e:
        print(f"⚠️ Open-Meteo error for {lat},{lon},{year}: {e}")
        return {}

# ------------------ 2️⃣ NASA POWER (Humidity) ------------------
def fetch_nasa_humidity(lat, lon, year):
    """Fetch mean relative humidity."""
    try:
        url = (
            f"https://power.larc.nasa.gov/api/temporal/daily/point?"
            f"parameters=RH2M"
            f"&latitude={lat}&longitude={lon}"
            f"&start={year}0101&end={year}1231"
            f"&community=AG&format=JSON"
        )
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        data = r.json().get("properties", {}).get("parameter", {})
        
        if "RH2M" in data:
            rh_values = [v for v in data["RH2M"].values() if v is not None and v != -999]
            if rh_values:
                return {"humidity_mean_(%)_(50x50km)": np.mean(rh_values)}
        
        return {"humidity_mean_(%)_(50x50km)": None}
    except Exception as e:
        print(f"⚠️ NASA humidity error {lat},{lon},{year}: {e}")
        return {"humidity_mean_(%)_(50x50km)": None}

# ------------------ 3️⃣ EARTH ENGINE (NDVI + Human coverage) ------------------
def fetch_gee_vars(lat, lon, year):
    """Fetch MODIS NDVI + WorldPop density."""
    import ee
    
    res = {
        "ndvi_mean_(0–1)_(0.25x0.25km)": None, 
        "human_density_(people/km²)_(0.1x0.1km)": None
    }
    
    try:
        point = ee.Geometry.Point(lon, lat)
        
        # NDVI
        ndvi_coll = (ee.ImageCollection("MODIS/061/MOD13Q1")
                     .filterDate(f"{year}-01-01", f"{year}-12-31")
                     .select("NDVI"))
        
        if ndvi_coll.size().getInfo() > 0:
            ndvi_mean = ndvi_coll.mean().reduceRegion(
                ee.Reducer.mean(), 
                point.buffer(5500), 
                250
            ).get("NDVI").getInfo()
            
            if ndvi_mean is not None:
                res["ndvi_mean_(0–1)_(0.25x0.25km)"] = ndvi_mean / 10000
    except Exception as e:
        print(f"⚠️ NDVI fail {lat},{lon},{year}: {e}")

    try:
        # WorldPop
        pop_coll = (ee.ImageCollection("WorldPop/GP/100m/pop")
                    .filterDate(f"{year}-01-01", f"{year}-12-31"))
        
        if pop_coll.size().getInfo() > 0:
            pop = pop_coll.mean().reduceRegion(
                ee.Reducer.mean(), 
                point.buffer(5500), 
                100
            ).get("population").getInfo()
            
            if pop is not None:
                res["human_density_(people/km²)_(0.1x0.1km)"] = pop
    except Exception as e:
        print(f"⚠️ WorldPop fail {lat},{lon},{year}: {e}")
    
    return res

# ------------------ 4️⃣ COMBINED WRAPPER WITH SMART BATCHING ------------------
def append_covariates(df, lat_col="lat_center", lon_col="lon_center", year_col="year", 
                     requests_per_minute=60, use_gee=True, 
                     cache_file="data/processed/climate_cache.json", 
                     checkpoint_file="data/processed/climate_checkpoint.csv",
                     checkpoint_interval=100):
    """
    Append climate covariates with proper rate limiting, caching, and checkpointing.
    
    Args:
        df: Input dataframe
        lat_col, lon_col, year_col: Column names
        requests_per_minute: Rate limit for API calls
        use_gee: Whether to fetch Google Earth Engine data (requires ee module)
        cache_file: JSON file to store API results cache
        checkpoint_file: CSV file to save progress checkpoints
        checkpoint_interval: Save checkpoint every N requests
    """
    df = df.copy()
    
    # Initialize columns
    cols = [
        "temp_mean_(°C)_(25x25km)",
        "precip_sum_(mm/year)_(25x25km)",
        "humidity_mean_(%)_(50x50km)",
        "ndvi_mean_(0–1)_(0.25x0.25km)",
        "human_density_(people/km²)_(0.1x0.1km)",
    ]
    for c in cols:
        df[c] = None
    
    # Load existing cache if available
    cache = {}
    if os.path.exists(cache_file):
        try:
            with open(cache_file, 'r') as f:
                cache_raw = json.load(f)
                # Convert string keys back to tuples
                cache = {eval(k): v for k, v in cache_raw.items()}
            print(f"✓ Loaded {len(cache)} entries from cache: {cache_file}")
        except Exception as e:
            print(f"⚠️ Could not load cache: {e}")
    
    # Get unique location-year combinations
    unique_rows = df[[lat_col, lon_col, year_col]].drop_duplicates()
    total = len(unique_rows)
    
    # Count how many we already have cached
    already_cached = sum(1 for _, row in unique_rows.iterrows() 
                        if (round(float(row[lat_col]), 3), 
                            round(float(row[lon_col]), 3), 
                            int(row[year_col])) in cache)
    
    print(f"Fetching data for {total} unique location-year combinations...")
    print(f"Already cached: {already_cached}, Need to fetch: {total - already_cached}")
    
    # Calculate delay between requests to respect rate limit
    delay = 60.0 / requests_per_minute
    
    def save_cache():
        """Save cache to JSON file."""
        try:
            # Convert tuple keys to strings for JSON
            cache_serializable = {str(k): v for k, v in cache.items()}
            with open(cache_file, 'w') as f:
                json.dump(cache_serializable, f, indent=2)
            print(f"💾 Cache saved to {cache_file}")
        except Exception as e:
            print(f"⚠️ Error saving cache: {e}")
    
    def save_checkpoint():
        """Save current progress to checkpoint file."""
        try:
            df_current = fill_dataframe_from_cache(df.copy(), cache, cols, 
                                                   lat_col, lon_col, year_col)
            df_current.to_csv( checkpoint_file, index=False)
            print(f"💾 Checkpoint saved to {checkpoint_file}")
        except Exception as e:
            print(f"⚠️ Error saving checkpoint: {e}")
    
    # Process each unique location-year
    try:
        for idx, (_, row) in enumerate(tqdm(unique_rows.iterrows(), total=total)):
            lat = round(float(row[lat_col]), 3)
            lon = round(float(row[lon_col]), 3)
            year = int(row[year_col])
            
            key = (lat, lon, year)
            
            if key in cache:
                continue
            
            # Fetch all data for this location
            result = {}
            
            # Open-Meteo (temp + precip)
            result.update(fetch_open_meteo_single(lat, lon, year))
            time.sleep(delay)
            
            # NASA (humidity)
            result.update(fetch_nasa_humidity(lat, lon, year))
            time.sleep(delay)
            
            # Google Earth Engine (NDVI + population) - if enabled
            if use_gee:
                try:
                    result.update(fetch_gee_vars(lat, lon, year))
                    time.sleep(delay)
                except Exception as e:
                    print(f"⚠️ GEE error (skipping): {e}")
                    result.update({
                        "ndvi_mean_(0–1)_(0.25x0.25km)": None,
                        "human_density_(people/km²)_(0.1x0.1km)": None
                    })
            else:
                result.update({
                    "ndvi_mean_(0–1)_(0.25x0.25km)": None,
                    "human_density_(people/km²)_(0.1x0.1km)": None
                })
            
            cache[key] = result
            
            # Save checkpoint at intervals
            if (idx + 1) % checkpoint_interval == 0:
                print(f"\n✓ Completed {idx + 1}/{total}. Saving checkpoint...")
                save_cache()
                save_checkpoint()
                time.sleep(2)  # Brief pause after checkpoint
            
            # Extra safety: pause every 50 requests
            elif (idx + 1) % 50 == 0:
                print(f"\n✓ Completed {idx + 1}/{total}. Pausing 5s...")
                save_cache()  # Save cache at regular intervals too
                time.sleep(5)
    
    except KeyboardInterrupt:
        print("\n⚠️ Interrupted by user. Saving progress...")
        save_cache()
        save_checkpoint()
        print("✓ Progress saved. You can resume by running the same command.")
        raise
    
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")
        print("Saving progress before exiting...")
        save_cache()
        save_checkpoint()
        raise
    
    # Final save
    print("\n📊 Filling dataframe with cached results...")
    save_cache()
    
    df = fill_dataframe_from_cache(df, cache, cols, lat_col, lon_col, year_col)
    
    return df


def fill_dataframe_from_cache(df, cache, cols, lat_col, lon_col, year_col):
    """Helper function to fill dataframe from cache."""
    for i, row in tqdm(df.iterrows(), total=len(df)):
        lat = round(float(row[lat_col]), 3)
        lon = round(float(row[lon_col]), 3)
        year = int(row[year_col])
        
        key = (lat, lon, year)
        vals = cache.get(key, {})
        
        for col in cols:
            if col in vals:
                df.at[i, col] = vals[col]
    
    return df


# ------------------ 5️⃣ EXAMPLE USAGE ------------------
if __name__ == "__main__":
    # Example with a small test dataset
    test_data = {
        'grid_id': ['g_0_321', 'g_0_322', 'g_0_323'],
        'year': [2018, 2018, 2018],
        'lat_center': [22.15, 22.25, 22.35],
        'lon_center': [60.05, 60.05, 60.05]
    }
    
    df_test = pd.DataFrame(test_data)
    
    init_earth_engine() # Initialize Earth Engine
    # Fetch without GEE (if you don't have Earth Engine set up)
    df_result = append_covariates(df_test, use_gee=True, requests_per_minute=30)
    
    print("\n✅ Results:")
    print(df_result)
    
    # Save results
    df_result.to_csv(os.path.join(base_path,"climate_data_test.csv"), index=False)
    print("\n💾 Saved to climate_data_test.csv")

Fetching data for 3 unique location-year combinations...
Already cached: 0, Need to fetch: 3


  0%|          | 0/3 [00:00<?, ?it/s]

⚠️ NASA humidity error 22.15,60.05,2018: 429 Client Error: Too Many Requests for url: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=RH2M&latitude=22.15&longitude=60.05&start=20180101&end=20181231&community=AG&format=JSON
⚠️ NDVI fail 22.15,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.
⚠️ WorldPop fail 22.15,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.


 33%|███▎      | 1/3 [00:07<00:15,  7.95s/it]

⚠️ NASA humidity error 22.25,60.05,2018: 429 Client Error: Too Many Requests for url: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=RH2M&latitude=22.25&longitude=60.05&start=20180101&end=20181231&community=AG&format=JSON
⚠️ NDVI fail 22.25,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.
⚠️ WorldPop fail 22.25,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.


...

⚠️ NASA humidity error 22.35,60.05,2018: 429 Client Error: Too Many Requests for url: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=RH2M&latitude=22.35&longitude=60.05&start=20180101&end=20181231&community=AG&format=JSON
⚠️ NDVI fail 22.35,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.
⚠️ WorldPop fail 22.35,60.05,2018: Earth Engine client library not initialized. See http://goo.gle/ee-auth.


100%|██████████| 3/3 [00:22<00:00,  7.39s/it]



📊 Filling dataframe with cached results...
💾 Cache saved to data/processed/climate_cache.json


100%|██████████| 3/3 [00:00<00:00, 751.35it/s]


✅ Results:
   grid_id  year  lat_center  lon_center temp_mean_(°C)_(25x25km)  \
0  g_0_321  2018       22.15       60.05                25.418904   
1  g_0_322  2018       22.25       60.05                25.506164   
2  g_0_323  2018       22.35       60.05                25.569863   

  precip_sum_(mm/year)_(25x25km) humidity_mean_(%)_(50x50km)  \
0                            4.2                        None   
1                            4.3                        None   
2                            5.4                        None   

  ndvi_mean_(0–1)_(0.25x0.25km) human_density_(people/km²)_(0.1x0.1km)  
0                          None                                   None  
1                          None                                   None  
2                          None                                   None  

💾 Saved to climate_data_test.csv


In [2]:
# First run (or resume from interruption)
df2 = append_covariates(
    df,
    use_gee=True,
    requests_per_minute=30,
    cache_file="data/processed/climate_cache.json",
    checkpoint_file="data/processed/climate_checkpoint.csv", 
    checkpoint_interval=100  # Save every 100 requests
)

# Save final results
df2.to_csv("data/processed/analysis_panel_causal_covariates.csv", index=False)

NameError: name 'df' is not defined